In [1]:
from sympy import *
import numpy as np
from sympy.plotting import plot3d,plot
from sympy import init_printing
#import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline
init_printing()

In [2]:
Variables=symbols("x1 y1 x2 y2")
x1,y1,x2,y2 = Variables
Time=symbols("t")
t=Time
Parameters = symbols("alpha beta gamma delta w a0 a1")
alpha,beta,gamma,delta,w,a0,a1 = Parameters

In [3]:
def VectorDivergence(vector,variables):
    D = []
    for vec in vector:
        row = []
        for var in variables:
            row.append(diff(vec,var))
        D.append(row)
    return Matrix(D)

In [4]:
F=Matrix([y1, gamma*cos(w*t) - delta*y1 - beta*x1 - alpha*x1**3,
          y2, gamma*cos(w*t) - delta*y2 - beta*x2 - alpha*x2**3])
F

⎡                y₁                ⎤
⎢                                  ⎥
⎢      3                           ⎥
⎢- α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(t⋅w)⎥
⎢                                  ⎥
⎢                y₂                ⎥
⎢                                  ⎥
⎢      3                           ⎥
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w)⎦

In [5]:
zeta=Matrix([0,0,
             0,0])
#zeta=eps*zeta
zeta

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [6]:
chi=Matrix([0,0,
            -(x2-x1*(a0+a1*sin(w*t))),-(y2-y1*(a0+a1*sin(w*t)))])

#chi=(epsilon1*sin(b*t)+epsilon2)*chi 
#chi=epsilon*chi
#epsilon=1
chi

⎡            0             ⎤
⎢                          ⎥
⎢            0             ⎥
⎢                          ⎥
⎢x₁⋅(a₀ + a₁⋅sin(t⋅w)) - x₂⎥
⎢                          ⎥
⎣y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂⎦

In [7]:
phi=Matrix([(x2-x1*(a0+a1*sin(w*t))),y2-y1*(a0+a1*sin(w*t))])
phi

⎡-x₁⋅(a₀ + a₁⋅sin(t⋅w)) + x₂⎤
⎢                           ⎥
⎣-y₁⋅(a₀ + a₁⋅sin(t⋅w)) + y₂⎦

In [8]:
N=VectorDivergence(phi,Variables)
N

⎡-a₀ - a₁⋅sin(t⋅w)          0          1  0⎤
⎢                                          ⎥
⎣        0          -a₀ - a₁⋅sin(t⋅w)  0  1⎦

In [9]:
# NF=(N*(F+zeta)).subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
# simplify(NF)
NF=N*(F+zeta)
NF
#simplify(NF)

⎡                                 y₁⋅(-a₀ - a₁⋅sin(t⋅w)) + y₂                 
⎢                                                                             
⎢      3                                                  ⎛      3            
⎣- α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w) + (-a₀ - a₁⋅sin(t⋅w))⋅⎝- α⋅x₁  - β⋅x₁ - δ⋅

                ⎤
                ⎥
               ⎞⎥
y₁ + γ⋅cos(t⋅w)⎠⎦

In [10]:
dPhi=diff(phi,Time)
dPhi

⎡-a₁⋅w⋅x₁⋅cos(t⋅w)⎤
⎢                 ⎥
⎣-a₁⋅w⋅y₁⋅cos(t⋅w)⎦

In [11]:
eqn=(NF+dPhi)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(eqn)

⎡                               -a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(a₀ + a₁⋅sin(t⋅w)) + y₂
⎢                                                                             
⎢                        3                                                 ⎛  
⎣-a₁⋅w⋅y₁⋅cos(t⋅w) - α⋅x₂  - β⋅x₂ - δ⋅y₂ + γ⋅cos(t⋅w) + (a₀ + a₁⋅sin(t⋅w))⋅⎝α⋅

                               ⎤
                               ⎥
  3                           ⎞⎥
x₁  + β⋅x₁ + δ⋅y₁ - γ⋅cos(t⋅w)⎠⎦

In [12]:
# This is working only because of the nature of the N matrix - be careful
zeta[2]=-(eqn)[0]
zeta[3]=-(eqn)[1]

simplify(zeta)

⎡                                                     0                       
⎢                                                                             
⎢                                                     0                       
⎢                                                                             
⎢                               a₁⋅w⋅x₁⋅cos(t⋅w) + y₁⋅(a₀ + a₁⋅sin(t⋅w)) - y₂ 
⎢                                                                             
⎢                       3                                                 ⎛   
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) - (a₀ + a₁⋅sin(t⋅w))⋅⎝α⋅x

                              ⎤
                              ⎥
                              ⎥
                              ⎥
                              ⎥
                              ⎥
 3                           ⎞⎥
₁  + β⋅x₁ + δ⋅y₁ - γ⋅cos(t⋅w)⎠⎦

In [13]:
NZ=N*zeta
NZ

⎡                                a₁⋅w⋅x₁⋅cos(t⋅w) - y₁⋅(-a₀ - a₁⋅sin(t⋅w)) - y
⎢                                                                             
⎢                       3                                                  ⎛  
⎣a₁⋅w⋅y₁⋅cos(t⋅w) + α⋅x₂  + β⋅x₂ + δ⋅y₂ - γ⋅cos(t⋅w) - (-a₀ - a₁⋅sin(t⋅w))⋅⎝- 

₂                                ⎤
                                 ⎥
    3                           ⎞⎥
α⋅x₁  - β⋅x₁ - δ⋅y₁ + γ⋅cos(t⋅w)⎠⎦

In [14]:
NF=N*(F+zeta)#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
simplify(NF)
simplify(NF+dPhi)

⎡0⎤
⎢ ⎥
⎣0⎦

In [15]:
del_zeta=VectorDivergence(zeta,Variables)
del_zeta

⎡                0                                    0                       
⎢                                                                             
⎢                0                                    0                       
⎢                                                                             
⎢          a₁⋅w⋅cos(t⋅w)                       a₀ + a₁⋅sin(t⋅w)               
⎢                                                                             
⎢                   ⎛        2    ⎞                                           
⎣(a₀ + a₁⋅sin(t⋅w))⋅⎝- 3⋅α⋅x₁  - β⎠  a₁⋅w⋅cos(t⋅w) - δ⋅(a₀ + a₁⋅sin(t⋅w))  3⋅α

  0       0 ⎤
            ⎥
  0       0 ⎥
            ⎥
  0       -1⎥
            ⎥
   2        ⎥
⋅x₂  + β  δ ⎦

In [16]:
del_chi=VectorDivergence(chi,Variables)
del_chi

⎡       0                 0          0   0 ⎤
⎢                                          ⎥
⎢       0                 0          0   0 ⎥
⎢                                          ⎥
⎢a₀ + a₁⋅sin(t⋅w)         0          -1  0 ⎥
⎢                                          ⎥
⎣       0          a₀ + a₁⋅sin(t⋅w)  0   -1⎦

In [17]:
del_F=VectorDivergence(F,Variables)
del_F

⎡      0        1         0        0 ⎤
⎢                                    ⎥
⎢        2                           ⎥
⎢- 3⋅α⋅x₁  - β  -δ        0        0 ⎥
⎢                                    ⎥
⎢      0        0         0        1 ⎥
⎢                                    ⎥
⎢                           2        ⎥
⎣      0        0   - 3⋅α⋅x₂  - β  -δ⎦

In [18]:
(del_F+del_zeta+del_chi)

⎡                0                                              1             
⎢                                                                             
⎢                  2                                                          
⎢          - 3⋅α⋅x₁  - β                                       -δ             
⎢                                                                             
⎢ a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w)                      a₀ + a₁⋅sin(t⋅w)      
⎢                                                                             
⎢                   ⎛        2    ⎞                                           
⎣(a₀ + a₁⋅sin(t⋅w))⋅⎝- 3⋅α⋅x₁  - β⎠  a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w) - δ⋅(a₀ 

                0   0 ⎤
                      ⎥
                      ⎥
                0   0 ⎥
                      ⎥
                -1  0 ⎥
                      ⎥
                      ⎥
+ a₁⋅sin(t⋅w))  0   -1⎦

In [19]:
Jac = N*(del_F+del_zeta+del_chi)*N.transpose()
#Jac=Jac.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])
Jac=simplify(Jac)
Jac#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])

⎡-(a₀ + a₁⋅sin(t⋅w))⋅(a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w)) - 1                   
⎢                                                                             
⎣                            0                               -(a₀ + a₁⋅sin(t⋅w

           0                             ⎤
                                         ⎥
))⋅(a₀ + a₁⋅w⋅cos(t⋅w) + a₁⋅sin(t⋅w)) - 1⎦

In [20]:
simplify(list(Jac.eigenvals().keys()))

AttributeError: 'list' object has no attribute 'replace'

In [ ]:
Ndot = diff(N,t)
Ndot

In [ ]:
Ndot*N.transpose()

In [ ]:

J = Jac+Ndot*N.transpose()
J = simplify(J)
J
J#.subs([(x2,x1*(a0+a1*sin(w*t))),(y2,y1*(a0+a1*sin(w*t))),(z2,z1*(a0+a1*sin(w*t)))])

In [ ]:
simplify(list(J.eigenvals().keys()))